In [1]:
import numpy as np
import re
import pandas as pd
from string import punctuation
import os
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [2]:
tweetdf = pd.read_csv(base+'loctrisma2016.csv')
tweetdf = tweetdf.sort_values(by='tweet_id')
tweetdf.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,tweet_id,created_at,from_user_id,source,text,to_user_id,to_user_name,...,location_name,country,location_full_name,location_type,country_code,collected_at,lga,state,lat,lng
375922,375922,412183,246385,682712842600972291,2016-01-01 00:00:00.000000,106082182,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@legallyfm @XFilesNews @thexfiles @mynewsaga &...,39023478.0,legallyfm,...,Glen Innes,Australia,"Glen Innes, New South Wales",city,AU,2013-01-01 00:00:01,Glen Innes Severn,NSW,-29.751838,151.736189
33932,33932,34218,503413,682712878915260418,2016-01-01 00:00:09.000000,269023896,"<a href=""http://sandaysoft.com/"" rel=""nofollow...",Temp: 33.1°C Wind: 5kph Pressure: 995.9hpa Fal...,NaN,NaN,...,Baradine,Australia,"Baradine, New South Wales",city,AU,2013-01-01 00:00:01,Warrumbungle Shire,NSW,-30.950000,149.066667
266436,266436,291148,502347,682712884565049344,2016-01-01 00:00:10.000000,379712969,"<a href=""http://sandaysoft.com/"" rel=""nofollow...","Wind 2.7 kts NE. Barometer 1017.9 mb, Falling ...",NaN,NaN,...,Raymond Terrace,Australia,"Raymond Terrace, New South Wales",city,AU,2013-01-01 00:00:01,Port Stephens,NSW,-32.754408,151.767512
132916,132916,145044,79326,682712982909005824,2016-01-01 00:00:33.000000,109140867,"<a href=""http://instagram.com"" rel=""nofollow"">...","Best coffee, food and swevice on the Hume and ...",NaN,NaN,...,Berremangra,Australia,"Berremangra, New South Wales",neighborhood,AU,2013-01-01 00:00:01,Hilltops,NSW,-34.830222,148.516892
565350,565350,623155,1323531,682713033836085248,2016-01-01 00:00:46.000000,178662965,"<a href=""http://twitter.com/download/android"" ...",Best start to 2016 - a morning at my favourite...,NaN,NaN,...,Dudley,Australia,"Dudley, Newcastle",neighborhood,AU,2013-01-01 00:00:01,Lake Macquarie,NSW,-32.990000,151.718000


In [3]:
by_lga = tweetdf.groupby(['lga'], as_index=False).agg({'text':lambda x:' '.join(x)})
by_lga

,lga,text
0,Albury,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...
1,Armidale Regional,Tranquility in the Park. @ Moore Park Inn Bout...
2,Ballina,Happy New Year to all my pals back home in the...
3,Balranald,Crackin night on the river. https://t.co/lGYg4...
4,Bathurst Regional,@ReneeYoungWWE Happy New Year to you &amp; you...
5,Bayside,@tamasys @mattRan wtf is maths circus? @toddjt...
6,Bega Valley,Suns out guns out. Coffee time. #crossfitgirls...
7,Bellingen,More from the Promised Lands #normanheywood #l...
8,Berrigan,@MoniqueAnnalyss hotttt 🔥🔥🔥 @MoniqueAnnalyss a...
9,Blacktown,Ahhh Tumblr. I currently have a hammock in the...


In [4]:
import string
import emoji
from unidecode import unidecode
from tokenizer import tokenizer
# Remove handles and urls but not hash tags.
# Handles might be useful for an is-reply feature but not for analysis of the text
smc_tokenizer = tokenizer.TweetTokenizer(preserve_handles=False, preserve_hashes=True, preserve_len=False,
                                         preserve_case=False, preserve_url=False)

In [5]:
# First, remove all emojis, then decode all unicode, then put emojis at the end
docs = []
for doc in by_lga['text']:
    emojis = ' '.join([c for c in doc if c in emoji.UNICODE_EMOJI])
    stripped = ''.join([c for c in doc if c not in emoji.UNICODE_EMOJI])
    decoded = unidecode(stripped)
    docs.append(' '.join([decoded, emojis]))
len(docs)

129

In [6]:
# TweetTokenizer weirdly keeps punctuation separated by space as one token:
puncts = set(string.punctuation + ' ')
docs = [smc_tokenizer.tokenize(doc) for doc in docs]
cleaned = []
for doc in docs:
    clean = [token for token in doc if not(len(token) == 1 and token not in emoji.UNICODE_EMOJI)
                and not(all(c in puncts for c in token))]
    cleaned.append(' '.join(clean))

In [7]:
cleaned[0]

"#2015bestnine #bestofnine2015 #byebye #2015 #hello #2016 the morey estate chances of relaunch 75 not likely and 25 for refresh #bushfooty #alburyboy hate this town the cleaner walks past needle in ya bum miserable piece shit when worked at subway was only person that stuck up what is with this shit hole town getting kick out of me having needle in my ass #purebreedsoffuckingidiots #albury never want to see ted just dumb thing to do out thinking through joke my new year resolution was to not getting shitty #couldntlastaday happy new year #nye2016 going to be smashing season this year good luck for the year tumut stormy weather rare night out without the kids almost date night with mrsk rare night out without the kids almost date night with mrsk beautiful little story new year resolution find another job so you don't have to put up waitress customers complaing on one man job #waittimes just wouldn't do this to anyone in my entire fucking life #lifestyleofthefuckwitsinfame i'm just grate

In [8]:
# Add cleaned tokens as an attribute to LGA:
cs = pd.Series(cleaned)
by_lga['cleaned'] = cs
by_lga

,lga,text,cleaned
0,Albury,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
1,Armidale Regional,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
2,Ballina,Happy New Year to all my pals back home in the...,happy new year to all my pals back home in the...
3,Balranald,Crackin night on the river. https://t.co/lGYg4...,crackin night on the river just posted photo o...
4,Bathurst Regional,@ReneeYoungWWE Happy New Year to you &amp; you...,happy new year to you your fam as well miss mi...
5,Bayside,@tamasys @mattRan wtf is maths circus? @toddjt...,wtf is maths circus sounds like it must've bee...
6,Bega Valley,Suns out guns out. Coffee time. #crossfitgirls...,suns out guns out coffee time #crossfitgirls #...
7,Bellingen,More from the Promised Lands #normanheywood #l...,more from the promised lands #normanheywood #l...
8,Berrigan,@MoniqueAnnalyss hotttt 🔥🔥🔥 @MoniqueAnnalyss a...,hottt appreciate your hotness monique it's 201...
9,Blacktown,Ahhh Tumblr. I currently have a hammock in the...,ahhh tumblr currently have hammock in the livi...


In [9]:
by_lga.to_csv('../../data/clean/trisma2016_cleaned_text_by_lga.csv')

In [10]:
# Not used here. This is just a backup in case this code gets misplaced.
def top_tfidf_feats(row, features, top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

#Return top features amongst documents in rows indentified by indices in grp_ids
def top_mean_feats(tfs, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    if grp_ids:
        D = tfs[grp_ids].toarray()
    else:
        D = tfs.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)